In [ ]:
"""
    EHCLO Project
    Copyright (C) 2025  Blaise CALMEL (INRAE)

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.
"""

'\n    EHCLO Project\n    Copyright (C) 2025  Blaise CALMEL (INRAE)\n\n    This program is free software: you can redistribute it and/or modify\n    it under the terms of the GNU General Public License as published by\n    the Free Software Foundation, either version 3 of the License, or\n    (at your option) any later version.\n\n    This program is distributed in the hope that it will be useful,\n    but WITHOUT ANY WARRANTY; without even the implied warranty of\n    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the\n    GNU General Public License for more details.\n\n    You should have received a copy of the GNU General Public License\n    along with this program.  If not, see <https://www.gnu.org/licenses/>.\n'

In [ ]:
import pyfiglet
ascii_banner = pyfiglet.figlet_format("EHCLO PROJECT")

In [ ]:
import pyfiglet
ascii_banner = pyfiglet.figlet_format("EHCLO PROJECT")

In [ ]:
print(f'################################ IMPORT & INITIALIZATION ################################', end='\n')

################################ IMPORT & INITIALIZATION ################################


In [ ]:
print(f'################################ IMPORT & INITIALIZATION ################################', end='\n')

################################ IMPORT & INITIALIZATION ################################


In [ ]:
print(f'> General imports...', end='\n')

> General imports...


In [ ]:
import sys
import os
import copy
# sys.path.insert(0, os.getcwd())
import time
import json

print(f'> Local imports...', end='\n')
from global_functions.load_data import *
from plot_functions.run_plot import *
from global_functions.format_data import *
from global_functions.shp_geometry import *
from global_functions.path_functions import  *
from global_functions.compute_narratives import compute_narratives

> Local imports...


In [ ]:
# Avoid crash with console when launched manually
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
plt.switch_backend('agg')

# Load environments variables
print(f'> Load json inputs...', end='\n')
with open('config.json') as config_file:
    config = json.load(config_file)

with open('files_setup.json') as files_setup:
    files_setup = json.load(files_setup)

settings_flatten = {}
for main_key in ['hydro_indicator', 'climate_indicator']:
    for key, value in files_setup[main_key].items():
        for subkey, subvalue in value.items():
            subvalue |= {'parent': key, 'type': main_key}
            settings_flatten |= {subkey: subvalue}

print(f'> Define paths...', end='\n')
dict_paths = define_paths(config)

### Files names
# Study folder
print(f'> Create output directories...', end='\n')
if not os.path.isdir(dict_paths['folder_study_results']):
    os.makedirs(dict_paths['folder_study_results'])

# Study figures folder
if not os.path.isdir(dict_paths['folder_study_figures']):
    os.makedirs(dict_paths['folder_study_figures'])

# Study data folder
if not os.path.isdir(dict_paths['folder_study_data']):
    os.makedirs(dict_paths['folder_study_data'])

if not os.path.isdir(dict_paths['folder_study_data'] + 'shapefiles'):
    os.makedirs(dict_paths['folder_study_data'] + 'shapefiles')

#%% LOAD STUDY REGION SHAPEFILE
print(f'################################ DEFINE STUDY AREA ################################', end='\n')
print(f'> Load shapefiles...', end='\n')
regions_shp, study_hydro_shp, study_climate_shp, rivers_shp = load_shp(dict_paths, files_setup)

# Check if study area is already matched with sim points
print(f'> Searching sim points in study area...', end='\n')
with open('reference_stations.json') as ref_stations:
    reference_stations = json.load(ref_stations)

flatten_reference_stations = {key: value for subdict in reference_stations.values() for key, value in subdict.items()}

for data_type, path in dict_paths['dict_study_points_sim'].items():
    if not os.path.isfile(path):
        print(f'>> Find {data_type} data points in study area')
        sim_all_points_info = open_shp(path_shp=dict_paths['dict_global_points_sim'][data_type])
        if data_type == 'hydro':
            overlay_shapefile(shapefile=study_hydro_shp, data=sim_all_points_info,
                              path_result=path, force_contains={'Suggesti_2': ['LA LOIRE', 'L\'ALLIER'],
                                                                'Suggestion': flatten_reference_stations.keys()})
        else:
            overlay_shapefile(shapefile=study_climate_shp, data=sim_all_points_info,
                              path_result=path)
    else:
        print(f'>> {data_type.capitalize()} data points already in the study area')

print(f'> Simplify shapefiles...', end='\n')
study_hydro_shp_simplified, study_climate_shp_simplified, study_rivers_shp_simplified, regions_shp_simplified, bounds = (
    simplify_shapefiles(study_hydro_shp, study_climate_shp, rivers_shp, regions_shp, tolerance=1000, zoom=1000))

hydro_sim_points_gdf_simplified = open_shp(path_shp=dict_paths['dict_study_points_sim']['hydro'])
hydro_sim_points_gdf_simplified = hydro_sim_points_gdf_simplified[hydro_sim_points_gdf_simplified['n'] >= 4]
hydro_sim_points_gdf_simplified = hydro_sim_points_gdf_simplified.reset_index(drop=True).set_index('Suggestion')
hydro_sim_points_gdf_simplified.index.names = ['name']

climate_sim_points_gdf = open_shp(path_shp=dict_paths['dict_study_points_sim']['climate'])
climate_sim_points_gdf_simplified = climate_sim_points_gdf.loc[
    climate_sim_points_gdf.groupby('name')['gid'].idxmin()].reset_index(drop=True)

all_years = files_setup["historical"] + [year for values in files_setup["horizons"].values() for
                                         year in values]
start_year = min(all_years)
end_year = max(all_years)

#%% NCDF Loading
load_ncdf = input("Load new NCDF ? (y/[n])")

if load_ncdf.lower().replace(" ", "") in ['y', 'yes']:
    print(f'################################ RUN OVER NCDF ################################', end='\n')
    # Get paths for selected sim
    print(f'> Load ncdf data paths...', end='\n')
    path_files = get_files_path(dict_paths=dict_paths, setup=files_setup)

    # Run among data type climate/hydro
    data_type='climate'
    subdict=path_files[data_type]
    rcp='rcp85'
    subdict2=subdict[rcp]
    indicator = "tasAdjust"
    paths = subdict2[indicator]
    for data_type, subdict in path_files.items():
        # Load simulation points for current data type
        # sim_points_gdf = open_shp(path_shp=dict_paths['dict_study_points_sim'][data_type])

        if data_type == "hydro":
            sim_points_gdf_simplified = hydro_sim_points_gdf_simplified
        else:
            sim_points_gdf_simplified = climate_sim_points_gdf_simplified
            # sim_points_gdf['weight'] = sim_points_gdf['surface'] / sim_points_gdf['total_surf']

        for rcp, subdict2 in subdict.items():
            for indicator, paths in subdict2.items():
                print(f'################################ RUN {data_type} {rcp} {indicator} ################################', end='\n')
                for name_indicator, settings_dict in files_setup[f'{data_type}_indicator'][indicator].items():
                    timestep = 'YE'
                    function = None
                    if 'timestep' in settings_dict:
                        timestep = settings_dict['timestep']

                    if 'extract_function' in settings_dict:
                        function = settings_dict['extract_function']

                    name_join = name_indicator.replace(" ", "-").replace(".", "")

                    path_ncdf = f"{dict_paths['folder_study_data']}{name_join}_{rcp}_{timestep}_{start_year}-{end_year}.nc"

                    if not os.path.isfile(path_ncdf):
                        print(f'> Create {indicator} export...', end='\n')
                        if len(paths) > 0 :
                            # paths = [
                            #     '/home/bcalmel/Documents/2_data/historical-rcp85/HadGEM2-ES/ALADIN63/ADAMONT/SMASH/debit_France_MOHC-HadGEM2-ES_historical-rcp85_r1i1p1_CNRM-ALADIN63_v3_MF-ADAMONT-SAFRAN-1980-2011_INRAE-SMASH_day_20050801-20990731.nc'
                            # ]
                            # paths_data=paths
                            # param_type=data_type
                            # sim_points_gdf=sim_points_gdf_simplified
                            #
                            # path_result=path_ncdf
                            # path_ncdf = f"{dict_paths['folder_study_data']}{name_join}_{rcp}_{timestep}_{start}-{end}.csv"
                            extract_ncdf_indicator(
                                paths_data=paths, param_type=data_type, sim_points_gdf=sim_points_gdf_simplified,
                                indicator=indicator, function=function, timestep=timestep,
                                start=start_year,
                                end=end_year,
                                path_result=path_ncdf,
                            )
                        else:
                            print(f'> Invalid {indicator} name', end='\n')
                    else:
                        print(f'> {path_ncdf} already exists', end='\n')

#%% Visualize results
narratives = None

> Load json inputs...
> Define paths...
> Create output directories...
################################ DEFINE STUDY AREA ################################
> Load shapefiles...


/home/bcalmel/Documents/1_codes/myvenv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/bcalmel/Documents/1_codes/myvenv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


> Searching sim points in study area...
>> Hydro data points already in the study area
>> Climate data points already in the study area
> Simplify shapefiles...
>> Simplify rivers...
>> Simplify regions background...
>> Simplify study area...


In [ ]:
stations=list(reference_stations['La Loire'].keys())

In [ ]:
files_setup=files_setup

In [ ]:
indictor_values=["QJXA", "QA", "VCN10"]

In [ ]:
threshold=0.8*len(reference_stations['La Loire'])

In [ ]:
narrative_method=None

In [ ]:
from sklearn.cluster import KMeans
import xarray as xr
import numpy as np
from sklearn.decomposition import PCA
from global_functions.format_data import format_dataset
from plot_functions.plot_narratives import plot_narratives

In [ ]:
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [ ]:
narratives = None

In [ ]:
def representative_item(X_cluster, centroids, cluster, cluster_id, indices_cluster, method='closest'):
    idx = None
    if method == 'closest':
        # Compute distance from cluster centroid
        distances = np.linalg.norm(X_cluster - centroids[cluster], axis=1)

        # Get index of the closest sim
        idx = indices_cluster[np.argmin(distances)]
    elif method == 'furthest':
        # Compute distance from other cluster centroids
        distances_list = []
        for c in cluster_id:
            if c != cluster:
                distances_list.append(np.linalg.norm(X_cluster - centroids[c], axis=1))

            distances = np.mean(distances_list, axis=0)
            idx = indices_cluster[np.argmax(distances)]
    elif method == 'combine':
        # Compute distance from cluster centroid
        distances_cluster = np.linalg.norm(X_cluster - centroids[cluster], axis=1)
        mask_cluster = distances_cluster < np.mean(distances_cluster)

        # Compute distance from other cluster centroids
        distances_list = []
        for c in cluster_id:
            if c != cluster:
                distances_list.append(np.linalg.norm(X_cluster - centroids[c], axis=1))
        distances_other = np.mean(distances_list, axis=0)
        idx = indices_cluster[mask_cluster][np.argmax(distances_other[mask_cluster])]

    return idx

In [ ]:
threshold=0

In [ ]:
# Load selected indicators
datasets_list = []
for indicator in indictor_values:
    # Open ncdf dataset
    path_ncdf = f"{dict_paths['folder_study_data']}{indicator}_rcp85_YE.nc"
    ds_stats  = xr.open_dataset(path_ncdf)

    # Compute stats
    ds_stats, var_names = format_dataset(ds=ds_stats, data_type='hydro', files_setup=files_setup)
    datasets_list.append(ds_stats)

data_arrays = []
datasets = [ds_i[var_names[f'simulation-horizon_by-sims_deviation']].sel(
    horizon='horizon3', gid=stations) for ds_i in datasets_list]
for i in range(len(datasets)):
    ds = datasets[i]
    for var_name, da in ds.data_vars.items():
        # Extract names part
        parts = var_name.split("_")
        nom_gcmrcm = "_".join(parts[:2])
        nom_bc, nom_hm = parts[2:4]

        # Generate new DataArray with sim as dimension
        da_expanded = da.expand_dims({
            # "indicator": [indicator_names[i]],
            "gcm-rcm": [nom_gcmrcm],
            # "rcm": [nom_rcm],
            "bc":  [nom_bc],
            "hm":  [nom_hm]
        })

        # Get name of the current indicator as var name
        da_expanded.name = indictor_values[i]

        data_arrays.append(da_expanded)

# Combine DataArrays
combined_da = xr.combine_by_coords(data_arrays)

# Count stations per sim
count_stations = combined_da[["QA"]].count(dim="gid")['QA'].values.flatten()

# Compute mean on selected stations
combined_da = combined_da.mean(dim='gid')

# # Weighted mean by cumulative distance between station
# gdf = hydro_sim_points_gdf_simplified.loc[stations]
# gdf["sum_distance"] = gdf.geometry.apply(lambda p: gdf.distance(p).sum())
# gdf["sum_distance"] = gdf["sum_distance"] / gdf["sum_distance"].mean()
#
# combined_da = combined_da.assign_coords(weights=("gid", gdf.reindex(ds["gid"].values)["sum_distance"].values))
# combined_da = combined_da.weighted(combined_da["weights"]).mean(dim="gid")

# Flatten dataset and generate new coordinate named "sample"
ds_stacked = combined_da.stack(sample=("gcm-rcm", "bc", "hm"))

# Generate matrix
X_imputed = np.column_stack([ds_stacked[var].values for var in indictor_values])

# KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_imputed)

# # Add labels to DataArray with sample dimension
# labels_da = xr.DataArray(labels, dims="sample", coords={"sample": ds_stacked.sample})

# # Unstack to same dimension as origin DataArray
# labels_unstacked = labels_da.unstack("sample")

# # Add labels as a new variable
# ds_clustered = combined_da.assign(cluster=labels_unstacked)

# Find centroids
centroids = kmeans.cluster_centers_  # de forme (n_clusters, n_features)

# Create mask for sim above threshold
above_threshold = count_stations > threshold
# Run on each cluster
cluster_id = np.unique(labels)
# Cluster info
# colors = plt.get_cmap("Dark2", 4).colors
# hex_colors = [mcolors.to_hex(c) for c in colors]

# # Load hm performances .fst files
# # Run once to install the related R packages
# utils = importr('utils')
# utils.install_packages('fst')
# utils.install_packages('data.table')

>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...
>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...


FileNotFoundError: [Errno 2] Aucun fichier ou dossier de ce nom: '/home/bcalmel/Documents/3_results/HMUC_Loire_Bretagne/data/VCN10_rcp85_YE.nc'

In [ ]:
cluster_names = ['A', 'B', 'C', 'D']

# Rank clusters
# ranks = np.argsort(np.argsort(centroids, axis=0), axis=0) + 1
# cumulative_ranks = ranks[:, 0] + ranks[:, 2]
# mask = np.ones(ranks.shape, dtype=bool)
#
# extreme = np.argmin(cumulative_ranks)
# mask[extreme, :] = False
# dry = np.argmin(np.where(mask, ranks, np.inf)[:, 2])
# mask[dry, :] = False
# flood = np.argmax(np.where(mask, ranks, -np.inf)[:, 0])
# mask[flood, :] = False
# last = np.argmin(np.where(mask, ranks, np.inf)[:, 2])
#
# narra_idx = [flood, dry, last, extreme]
# narra_idx = [1, 2, 0, 3]
hex_colors = ["#016367", "#9E3A14", "#E66912", "#0B1C48"]
# Bleunavy Orange Brun Turquoise https://www.canva.com/colors/color-palettes/freshly-sliced-fruit/
# hex_colors = [hex_colors[i] for i in narra_idx]

rows = None
if narrative_method is None:
    methods = ['closest', 'furthest', 'combine']
    rows = ['Proche', 'Lointain', 'Mixte']
else:
    methods = [narrative_method]
meth_list = []
for narrative_method in methods:
    representative_groups = {}
    for cluster in cluster_id:
        # Index of cluster values
        indices_cluster = np.where(labels == cluster)[0]

        # Filter indices for sim above threshold
        indices_mask = above_threshold[indices_cluster]
        if len(indices_mask) > 0:
            indices_cluster = indices_cluster[indices_mask]

        # Get vector of these sims
        X_cluster = X_imputed[indices_cluster, :]

        idx = representative_item(X_cluster, centroids, cluster, cluster_id, indices_cluster, method=narrative_method)

        # Extract coordinate (gcm-rcm, bc, hm) of selected sim
        coords_gcm_rcm = ds_stacked["gcm-rcm"].isel(sample=idx).values
        coords_bc      = ds_stacked["bc"].isel(sample=idx).values
        coords_hm      = ds_stacked["hm"].isel(sample=idx).values

        # Save result in dict
        representative_groups[cluster] = {
            "gcm-rcm": coords_gcm_rcm,
            "bc": coords_bc,
            "hm": coords_hm,
            # "distance": distances[np.argmin(distances)],
            "idx": idx,
            "color": hex_colors[cluster],
            "name": cluster_names[cluster],
            "method": narrative_method
        }
    meth_list.append(representative_groups)

NameError: name 'cluster_id' is not defined

In [ ]:
cluster_names = ['A', 'B', 'C', 'D']

In [ ]:
cluster_names = ['A', 'B', 'C', 'D']

In [ ]:
hex_colors = ["#016367", "#9E3A14", "#E66912", "#0B1C48"]

In [ ]:
rows = None
if narrative_method is None:
    methods = ['closest', 'furthest', 'combine']
    rows = ['Proche', 'Lointain', 'Mixte']
else:
    methods = [narrative_method]
meth_list = []

In [ ]:
methods

['closest']

In [ ]:
methods = ['closest', 'furthest', 'combine']

In [ ]:
rows = ['Proche', 'Lointain', 'Mixte']

In [ ]:
cluster_id

NameError: name 'cluster_id' is not defined

In [ ]:
above_threshold

NameError: name 'above_threshold' is not defined

In [ ]:
kmeans

NameError: name 'kmeans' is not defined

In [ ]:
data_arrays = []
datasets = [ds_i[var_names[f'simulation-horizon_by-sims_deviation']].sel(
    horizon='horizon3', gid=stations) for ds_i in datasets_list]
for i in range(len(datasets)):
    ds = datasets[i]
    for var_name, da in ds.data_vars.items():
        # Extract names part
        parts = var_name.split("_")
        nom_gcmrcm = "_".join(parts[:2])
        nom_bc, nom_hm = parts[2:4]

        # Generate new DataArray with sim as dimension
        da_expanded = da.expand_dims({
            # "indicator": [indicator_names[i]],
            "gcm-rcm": [nom_gcmrcm],
            # "rcm": [nom_rcm],
            "bc":  [nom_bc],
            "hm":  [nom_hm]
        })

        # Get name of the current indicator as var name
        da_expanded.name = indictor_values[i]

        data_arrays.append(da_expanded)

# Combine DataArrays
combined_da = xr.combine_by_coords(data_arrays)

# Count stations per sim
count_stations = combined_da[["QA"]].count(dim="gid")['QA'].values.flatten()

# Compute mean on selected stations
combined_da = combined_da.mean(dim='gid')

# # Weighted mean by cumulative distance between station
# gdf = hydro_sim_points_gdf_simplified.loc[stations]
# gdf["sum_distance"] = gdf.geometry.apply(lambda p: gdf.distance(p).sum())
# gdf["sum_distance"] = gdf["sum_distance"] / gdf["sum_distance"].mean()
#
# combined_da = combined_da.assign_coords(weights=("gid", gdf.reindex(ds["gid"].values)["sum_distance"].values))
# combined_da = combined_da.weighted(combined_da["weights"]).mean(dim="gid")

# Flatten dataset and generate new coordinate named "sample"
ds_stacked = combined_da.stack(sample=("gcm-rcm", "bc", "hm"))

# Generate matrix
X_imputed = np.column_stack([ds_stacked[var].values for var in indictor_values])

# KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_imputed)

# # Add labels to DataArray with sample dimension
# labels_da = xr.DataArray(labels, dims="sample", coords={"sample": ds_stacked.sample})

# # Unstack to same dimension as origin DataArray
# labels_unstacked = labels_da.unstack("sample")

# # Add labels as a new variable
# ds_clustered = combined_da.assign(cluster=labels_unstacked)

# Find centroids
centroids = kmeans.cluster_centers_  # de forme (n_clusters, n_features)

# Create mask for sim above threshold
above_threshold = count_stations > threshold
# Run on each cluster
cluster_id = np.unique(labels)
# Cluster info
# colors = plt.get_cmap("Dark2", 4).colors
# hex_colors = [mcolors.to_hex(c) for c in colors]

# # Load hm performances .fst files
# # Run once to install the related R packages
# utils = importr('utils')
# utils.install_packages('fst')
# utils.install_packages('data.table')

KeyError: "No variable named 'VCN10'. Variables on the dataset include ['QA', 'horizon', 'QJXA', 'sample', 'gcm-rcm', 'bc', 'hm']"

In [ ]:
data_arrays = []
datasets = [ds_i[var_names[f'simulation-horizon_by-sims_deviation']].sel(
    horizon='horizon3', gid=stations) for ds_i in datasets_list]
for i in range(len(datasets)):
    ds = datasets[i]
    for var_name, da in ds.data_vars.items():
        # Extract names part
        parts = var_name.split("_")
        nom_gcmrcm = "_".join(parts[:2])
        nom_bc, nom_hm = parts[2:4]

        # Generate new DataArray with sim as dimension
        da_expanded = da.expand_dims({
            # "indicator": [indicator_names[i]],
            "gcm-rcm": [nom_gcmrcm],
            # "rcm": [nom_rcm],
            "bc":  [nom_bc],
            "hm":  [nom_hm]
        })

        # Get name of the current indicator as var name
        da_expanded.name = indictor_values[i]

        data_arrays.append(da_expanded)

In [ ]:
# Combine DataArrays
combined_da = xr.combine_by_coords(data_arrays)

# Count stations per sim
count_stations = combined_da[["QA"]].count(dim="gid")['QA'].values.flatten()

# Compute mean on selected stations
combined_da = combined_da.mean(dim='gid')

In [ ]:
# Flatten dataset and generate new coordinate named "sample"
ds_stacked = combined_da.stack(sample=("gcm-rcm", "bc", "hm"))

# Generate matrix
X_imputed = np.column_stack([ds_stacked[var].values for var in indictor_values])

# KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_imputed)

KeyError: "No variable named 'VCN10'. Variables on the dataset include ['QA', 'horizon', 'QJXA', 'sample', 'gcm-rcm', 'bc', 'hm']"

In [ ]:
start_year

1991

In [ ]:
end_year

2099

In [ ]:
# Load selected indicators
datasets_list = []
for indicator in indictor_values:
    # Open ncdf dataset
    path_ncdf = f"{dict_paths['folder_study_data']}{indicator}_rcp85_YE_{start_year}-{end_year}.nc"
    ds_stats  = xr.open_dataset(path_ncdf)

    # Compute stats
    ds_stats, var_names = format_dataset(ds=ds_stats, data_type='hydro', files_setup=files_setup)
    datasets_list.append(ds_stats)

>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...
>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...
>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...


In [ ]:
data_arrays = []
datasets = [ds_i[var_names[f'simulation-horizon_by-sims_deviation']].sel(
    horizon='horizon3', gid=stations) for ds_i in datasets_list]
for i in range(len(datasets)):
    ds = datasets[i]
    for var_name, da in ds.data_vars.items():
        # Extract names part
        parts = var_name.split("_")
        nom_gcmrcm = "_".join(parts[:2])
        nom_bc, nom_hm = parts[2:4]

        # Generate new DataArray with sim as dimension
        da_expanded = da.expand_dims({
            # "indicator": [indicator_names[i]],
            "gcm-rcm": [nom_gcmrcm],
            # "rcm": [nom_rcm],
            "bc":  [nom_bc],
            "hm":  [nom_hm]
        })

        # Get name of the current indicator as var name
        da_expanded.name = indictor_values[i]

        data_arrays.append(da_expanded)

KeyError: "not all values found in index 'gid'"

In [ ]:
ds_stats

<xarray.Dataset> Size: 14MB
Dimensions:                                                         (time: 109,
                                                                     gid: 66,
                                                                     horizon: 4)
Coordinates:
  * time                                                            (time) datetime64[ns] 872B ...
  * gid                                                             (gid) |S10 660B ...
  * horizon                                                         (horizon) <U10 160B ...
    x                                                               (gid) float64 528B ...
    y                                                               (gid) float64 528B ...
    historical                                                      (time) bool 109B ...
    horizon1                                                        (time) bool 109B ...
    horizon2                                                        (time) bool 109B ...
    horizon3                                                        (time) bool 109B ...
Data variables: (12/1116)
    CNRM-CM5_ALADIN63_ADAMONT_CTRIP                                 (time, gid) float32 29kB ...
    CNRM-CM5_ALADIN63_ADAMONT_EROS                                  (time, gid) float32 29kB ...
    CNRM-CM5_ALADIN63_ADAMONT_GRSD                                  (time, gid) float32 29kB ...
    CNRM-CM5_ALADIN63_ADAMONT_J2000                                 (time, gid) float32 29kB ...
    CNRM-CM5_ALADIN63_ADAMONT_MORDOR-SD                             (time, gid) float32 29kB ...
    CNRM-CM5_ALADIN63_ADAMONT_MORDOR-TS                             (time, gid) float32 29kB ...
    ...                                                              ...
    horizon_deviation_SMASH-median                                  (horizon, gid) float32 1kB ...
    horizon_deviation_SMASH-mean                                    (horizon, gid) float32 1kB ...
    timeline-deviation-median                                       (time, gid) float32 29kB ...
    timeline-deviation-mean                                         (time, gid) float32 29kB ...
    timeline-difference-median                                      (time, gid) float32 29kB ...
    timeline-difference-mean                                        (time, gid) float32 29kB ...

In [ ]:
# Load selected indicators
datasets_list = []
for indicator in indictor_values:
    # Open ncdf dataset
    path_ncdf = f"{dict_paths['folder_study_data']}{indicator}_rcp85_YE_{start_year}-{end_year}.nc"
    ds_stats  = xr.open_dataset(path_ncdf)

    # Compute stats
    ds_stats, var_names = format_dataset(ds=ds_stats, data_type='hydro', files_setup=files_setup)
    ds_stats['gid'] = ds_stats['gid'].astype(str)
    datasets_list.append(ds_stats)

data_arrays = []

>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...
>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...
>> Define horizons...
>> Compute mean by horizon...
>> Compute deviation & difference by horizon for each simulation...
>> Compute stats by horizon among simulations...
>> Compute stats by HM by horizon among simulations [difference]...
>> Compute stats by HM by horizon among simulations [deviation]...


In [ ]:
data_arrays = []
datasets = [ds_i[var_names[f'simulation-horizon_by-sims_deviation']].sel(
    horizon='horizon3', gid=stations) for ds_i in datasets_list]

In [ ]:
for i in range(len(datasets)):
    ds = datasets[i]
    for var_name, da in ds.data_vars.items():
        # Extract names part
        parts = var_name.split("_")
        nom_gcmrcm = "_".join(parts[:2])
        nom_bc, nom_hm = parts[2:4]

        # Generate new DataArray with sim as dimension
        da_expanded = da.expand_dims({
            # "indicator": [indicator_names[i]],
            "gcm-rcm": [nom_gcmrcm],
            # "rcm": [nom_rcm],
            "bc":  [nom_bc],
            "hm":  [nom_hm]
        })

        # Get name of the current indicator as var name
        da_expanded.name = indictor_values[i]

        data_arrays.append(da_expanded)

In [ ]:
# Combine DataArrays
combined_da = xr.combine_by_coords(data_arrays)

# Count stations per sim
count_stations = combined_da[["QA"]].count(dim="gid")['QA'].values.flatten()

# Compute mean on selected stations
combined_da = combined_da.mean(dim='gid')

In [ ]:
# Flatten dataset and generate new coordinate named "sample"
ds_stacked = combined_da.stack(sample=("gcm-rcm", "bc", "hm"))

# Generate matrix
X_imputed = np.column_stack([ds_stacked[var].values for var in indictor_values])

# KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_imputed)

# # Add labels to DataArray with sample dimension
# labels_da = xr.DataArray(labels, dims="sample", coords={"sample": ds_stacked.sample})

# # Unstack to same dimension as origin DataArray
# labels_unstacked = labels_da.unstack("sample")

# # Add labels as a new variable
# ds_clustered = combined_da.assign(cluster=labels_unstacked)

# Find centroids
centroids = kmeans.cluster_centers_  # de forme (n_clusters, n_features)

# Create mask for sim above threshold
above_threshold = count_stations > threshold
# Run on each cluster
cluster_id = np.unique(labels)

In [ ]:
cluster_names = ['A', 'B', 'C', 'D']

In [ ]:
hex_colors = ["#016367", "#9E3A14", "#E66912", "#0B1C48"]
# Bleunavy Orange Brun Turquoise https://www.canva.com/colors/color-palettes/freshly-sliced-fruit/
# hex_colors = [hex_colors[i] for i in narra_idx]

rows = None
if narrative_method is None:
    methods = ['closest', 'furthest', 'combine']
    rows = ['Proche', 'Lointain', 'Mixte']
else:
    methods = [narrative_method]
meth_list = []

In [ ]:
methods

['closest']

In [ ]:
narrative_method

'closest'

In [ ]:
narrative_method

'closest'

In [ ]:
narrative_method = ['closest', 'furthest', 'combine']

In [ ]:
methods = [narrative_method]

In [ ]:
methods

[['closest', 'furthest', 'combine']]

In [ ]:
methods = ['closest', 'furthest', 'combine']

In [ ]:
rows = ['Proche', 'Lointain', 'Mixte']

In [ ]:
methods = ['closest', 'furthest', 'combine']

In [ ]:
rows = ['Proche', 'Lointain', 'Mixte']

In [ ]:
meth_list = []

In [ ]:
for narrative_method in methods:
    representative_groups = {}
    for cluster in cluster_id:
        # Index of cluster values
        indices_cluster = np.where(labels == cluster)[0]

        # Filter indices for sim above threshold
        indices_mask = above_threshold[indices_cluster]
        if len(indices_mask) > 0:
            indices_cluster = indices_cluster[indices_mask]

        # Get vector of these sims
        X_cluster = X_imputed[indices_cluster, :]

        idx = representative_item(X_cluster, centroids, cluster, cluster_id, indices_cluster, method=narrative_method)

        # Extract coordinate (gcm-rcm, bc, hm) of selected sim
        coords_gcm_rcm = ds_stacked["gcm-rcm"].isel(sample=idx).values
        coords_bc      = ds_stacked["bc"].isel(sample=idx).values
        coords_hm      = ds_stacked["hm"].isel(sample=idx).values

        # Save result in dict
        representative_groups[cluster] = {
            "gcm-rcm": coords_gcm_rcm,
            "bc": coords_bc,
            "hm": coords_hm,
            # "distance": distances[np.argmin(distances)],
            "idx": idx,
            "color": hex_colors[cluster],
            "name": cluster_names[cluster],
            "method": narrative_method
        }
    meth_list.append(representative_groups)

/home/bcalmel/Documents/1_codes/myvenv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/bcalmel/Documents/1_codes/myvenv/lib/python3.10/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
meth_list

[{np.int32(0): {'gcm-rcm': array('EC-EARTH_RCA4', dtype='<U13'),
   'bc': array('ADAMONT', dtype='<U7'),
   'hm': array('GRSD', dtype='<U4'),
   'idx': np.int64(38),
   'color': '#016367',
   'name': 'A',
   'method': 'closest'},
  np.int32(1): {'gcm-rcm': array('IPSL-CM5A-MR_RCA4', dtype='<U17'),
   'bc': array('ADAMONT', dtype='<U7'),
   'hm': array('ORCHIDEE', dtype='<U8'),
   'idx': np.int64(96),
   'color': '#9E3A14',
   'name': 'B',
   'method': 'closest'},
  np.int32(2): {'gcm-rcm': array('CNRM-CM5_ALADIN63', dtype='<U17'),
   'bc': array('ADAMONT', dtype='<U7'),
   'hm': array('J2000', dtype='<U5'),
   'idx': np.int64(3),
   'color': '#E66912',
   'name': 'C',
   'method': 'closest'},
  np.int32(3): {'gcm-rcm': array('NorESM1-M_REMO', dtype='<U14'),
   'bc': array('ADAMONT', dtype='<U7'),
   'hm': array('J2000', dtype='<U5'),
   'idx': np.int64(138),
   'color': '#0B1C48',
   'name': 'D',
   'method': 'closest'}},
 {np.int32(0): {'gcm-rcm': array('HadGEM2-ES_CCLM4-8-17', dtype=

In [ ]:
narratives = {methods[i] : {f"{value['gcm-rcm']}_{value['bc']}_{value['hm']}": {'color': value['color'], 'zorder': 10,
                                                                   'label': f"{value['name'].title()}", # [{value['gcm-rcm']}_{value['bc']}_{value['hm']}]",
                                                                   'linewidth': 1} for key, value in rp.items()} for i, rp in enumerate(meth_list)}

In [ ]:
narratives

{'closest': {'EC-EARTH_RCA4_ADAMONT_GRSD': {'color': '#016367',
   'zorder': 10,
   'label': 'A',
   'linewidth': 1},
  'IPSL-CM5A-MR_RCA4_ADAMONT_ORCHIDEE': {'color': '#9E3A14',
   'zorder': 10,
   'label': 'B',
   'linewidth': 1},
  'CNRM-CM5_ALADIN63_ADAMONT_J2000': {'color': '#E66912',
   'zorder': 10,
   'label': 'C',
   'linewidth': 1},
  'NorESM1-M_REMO_ADAMONT_J2000': {'color': '#0B1C48',
   'zorder': 10,
   'label': 'D',
   'linewidth': 1}},
 'furthest': {'HadGEM2-ES_CCLM4-8-17_ADAMONT_MORDOR-SD': {'color': '#016367',
   'zorder': 10,
   'label': 'A',
   'linewidth': 1},
  'IPSL-CM5A-MR_HIRHAM5_ADAMONT_MORDOR-TS': {'color': '#9E3A14',
   'zorder': 10,
   'label': 'B',
   'linewidth': 1},
  'NorESM1-M_WRF381P_ADAMONT_ORCHIDEE': {'color': '#E66912',
   'zorder': 10,
   'label': 'C',
   'linewidth': 1},
  'NorESM1-M_WRF381P_ADAMONT_SIM2': {'color': '#0B1C48',
   'zorder': 10,
   'label': 'D',
   'linewidth': 1}},
 'combine': {'EC-EARTH_HadREM3-GA7_ADAMONT_SMASH': {'color': '#0163

In [ ]:
narratives.keys()

dict_keys(['closest', 'furthest', 'combine'])

In [ ]:
narratives['combine']

{'EC-EARTH_HadREM3-GA7_ADAMONT_SMASH': {'color': '#016367',
  'zorder': 10,
  'label': 'A',
  'linewidth': 1},
 'IPSL-CM5A-MR_HIRHAM5_ADAMONT_SMASH': {'color': '#9E3A14',
  'zorder': 10,
  'label': 'B',
  'linewidth': 1},
 'MPI-ESM-LR_RegCM4-6_ADAMONT_J2000': {'color': '#E66912',
  'zorder': 10,
  'label': 'C',
  'linewidth': 1},
 'NorESM1-M_WRF381P_ADAMONT_CTRIP': {'color': '#0B1C48',
  'zorder': 10,
  'label': 'D',
  'linewidth': 1}}

In [ ]:
narratives['furthest']

{'HadGEM2-ES_CCLM4-8-17_ADAMONT_MORDOR-SD': {'color': '#016367',
  'zorder': 10,
  'label': 'A',
  'linewidth': 1},
 'IPSL-CM5A-MR_HIRHAM5_ADAMONT_MORDOR-TS': {'color': '#9E3A14',
  'zorder': 10,
  'label': 'B',
  'linewidth': 1},
 'NorESM1-M_WRF381P_ADAMONT_ORCHIDEE': {'color': '#E66912',
  'zorder': 10,
  'label': 'C',
  'linewidth': 1},
 'NorESM1-M_WRF381P_ADAMONT_SIM2': {'color': '#0B1C48',
  'zorder': 10,
  'label': 'D',
  'linewidth': 1}}

In [ ]:
fst = importr('fst')
dt = importr('data.table')

# Read the .fst file
df = fst.read_fst(f"/home/bcalmel/Documents/2_data/hydrologie/dataEX_Explore2_criteria_diagnostic_BF.fst")
# Convert to pandas dataframe
with (ro.default_converter + pandas2ri.converter).context():
    df = ro.conversion.get_conversion().rpy2py(df)

R[write to console]: fstcore package v0.10.0

R[write to console]: (OpenMP detected, using 8 threads)

R[write to console]: (OpenMP detected, using 8 threads)



In [ ]:
df

,HM,code,BFI_obs,BFI_sim,BFI,BFM_obs,BFM_sim,BFM,med{debutBF}_obs,med{debutBF}_sim,...,med{finBF},med{dtBF}_obs,med{dtBF}_sim,med{dtBF},med{vBF}_obs,med{vBF}_sim,med{vBF},med{dtRec}_obs,med{dtRec}_sim,med{dtRec}
1,CTRIP,A105003001,0.512885,0.400396,-0.219325,0.687138,0.769799,0.120298,303.000000,306.000000,...,-0.078455,237.0,223.0,-0.059072,38.462396,47.341512,0.230852,12.177512,13.450061,0.104500
2,CTRIP,A107020001,0.529975,0.598879,0.130013,0.838780,0.964084,0.149387,312.000000,346.186652,...,-0.140942,192.0,130.5,-0.320312,9.555553,4.089516,-0.572027,13.563254,31.910773,1.352737
3,CTRIP,A204010101,0.537787,0.475704,-0.115441,0.812701,0.867943,0.067974,318.000000,313.000000,...,-0.386445,196.0,189.0,-0.035714,87.665663,98.915788,0.128330,14.270081,13.478071,-0.055501
4,CTRIP,A212020002,0.648113,0.472274,-0.271310,0.742893,0.879715,0.184174,308.000000,311.000000,...,-1.016222,223.0,193.0,-0.134529,50.825049,21.195538,-0.582971,18.379946,14.247070,-0.224858
5,CTRIP,A231020001,0.495506,0.530087,0.069790,0.917208,0.804867,-0.122482,320.000000,306.500000,...,1.100667,169.0,195.5,0.156805,19.384972,21.573267,0.112886,13.371817,20.864768,0.560354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,SMASH,Y662000301,0.725330,0.641028,-0.116226,0.615813,0.624669,0.014381,289.784701,288.627766,...,-0.061402,264.0,265.0,0.003788,224.529026,193.856471,-0.136608,28.907246,18.819661,-0.348964
3617,SMASH,Y700000201,0.465529,0.309885,-0.334338,0.953053,0.870723,-0.086386,320.500000,305.500000,...,0.088092,170.0,195.0,0.147059,43.263292,28.001995,-0.352754,12.987078,6.996314,-0.461287
3618,SMASH,Y862000101,0.635873,0.570752,-0.102412,0.872170,0.849606,-0.025870,321.000000,318.000000,...,0.051906,185.0,185.0,0.000000,110.099719,94.244538,-0.144007,20.829845,16.904144,-0.188465
3619,SMASH,Y881000102,0.531725,0.357359,-0.327926,0.894443,0.914984,0.022965,323.000000,320.500000,...,-0.424643,182.0,181.0,-0.005495,56.160108,39.234713,-0.301378,17.854666,9.537679,-0.465816
